This runs an interactive form of ENFORMER. Does not use GPUs.

In [1]:
import json, os, sys, h5py, subprocess, tqdm, json
import pandas as pd
import numpy as np
import importlib
from functools import lru_cache

import cyvcf2

In [2]:
os.getcwd()

'/lus/theta-fs0/projects/covid-ct/imlab/users/temi/projects/TFXcan/enformer-predict/scripts-2'

In [25]:
script_path = os.getcwd()
sys.path.append(f'{script_path}/utilities')

import runPredictionUtilities
import enformerUsageCodes

# this helps reload the modules incase changes were made
importlib.reload(enformerUsageCodes)
importlib.reload(runPredictionUtilities)

<module 'runPredictionUtilities' from '/lus/theta-fs0/projects/covid-ct/imlab/users/temi/projects/TFXcan/enformer-predict/scripts-2/utilities/runPredictionUtilities.py'>

In [7]:
# read the parameters file
with open(f'{script_path}/../metadata/enformer_parameters.json') as f:

    parameters = json.load(f)

    intervals_dir = parameters['interval_list_dir']
    model_path = parameters['model_path']
    fasta_file = parameters['hg38_fasta_file']
    output_dir = parameters['output_dir']
    individuals = parameters['individuals']
    vcf_file = parameters['vcf_file']
    path_to_bcftools = parameters['path_to_bcftools']
    path_to_tabix = parameters['path_to_tabix']
    temporary_vcf_dir = parameters['temporary_vcf_dir']
    TF = parameters['TF']
    logfile_path = parameters['logfile_path']

In [4]:
logfile_path = "/projects/covid-ct/imlab/users/temi/projects/TFXcan/enformer-predict/predictions-log"
individuals = ['LuCaP_70'] #'LuCaP_167' #LuCaP_208' #'LuCaP_145'

each_individual = individuals[0]

In [5]:
logfile_csv = f'{logfile_path}/{each_individual}_predictions_log.csv'
if os.path.isfile(logfile_csv):
    logfile = pd.read_csv(logfile_csv)
    open_mode = 'a'
else:
    logfile = None
    open_mode = 'w'

logfile

,motif,individual,status,sequence_type
0,chr6_91587723_91587732,LuCaP_70,completed,var
1,chr4_80390310_80390319,LuCaP_70,completed,var
2,chr18_36661671_36661680,LuCaP_70,completed,var
3,chr4_71383885_71383894,LuCaP_70,completed,var
4,chr1_214605892_214605901,LuCaP_70,completed,var
...,...,...,...,...
97,chr1_57276627_57276636,LuCaP_70,completed,var
98,chr3_63673748_63673757,LuCaP_70,completed,var
99,chr5_104459392_104459401,LuCaP_70,completed,var
100,chr1_214605892_214605901,LuCaP_70,completed,var


In [23]:
a = pd.read_table(f'{intervals_dir}/{each_individual}_{TF}_400000.txt', sep=' ', header=None)
list_of_regions = a[0].tolist()[0:6] # a list of queries
list_of_regions

['chr1_214605892_214605901',
 'chr6_91587723_91587732',
 'chr11_106849851_106849860',
 'chr4_181081456_181081465',
 'chr4_80390310_80390319',
 'chr18_36661671_36661680']

In [11]:
if not os.path.exists(f'{output_dir}/{each_individual}'):
    print(f'[INFO] Creating output directory at {output_dir}/{each_individual}')
    os.makedirs(f'{output_dir}/{each_individual}')

In [13]:
@lru_cache(1)
def make_cyvcf_object(vcf_file=vcf_file, sample=each_individual):
    import cyvcf2
    return(cyvcf2.cyvcf2.VCF(vcf_file, samples=sample))

@lru_cache(1)
def get_model(model_class, model_path=model_path):
    return model_class(model_path)

fasta_object = enformerUsageCodes.FastaStringExtractor(fasta_file)
fasta_object

In [16]:
input_sequences = [runPredictionUtilities.create_individual_input_for_enformer(region=query, individual=each_individual, vcf_func=make_cyvcf_object, fasta_object=fasta_object, hap_type = 'hap1', resize_for_enformer=True, resize_length=None) for query in tqdm.tqdm(list_of_regions)]

100%|██████████| 201/201 [00:22<00:00,  8.75it/s]


In [ ]:
[runPredictionUtilities.enformer_predict(input_sequence['sequence'], region=input_sequence['region'], sample=each_individual, seq_type=input_sequence['sequence_source'], model_path=model_path, model_func=get_model, output_dir=output_dir, script_path=script_path, logfile_path=logfile_path) for input_sequence in input_sequences]

Or

In [45]:
out = [runPredictionUtilities.run_single_predictions(region=query, individual=each_individual, vcf_func=make_cyvcf_object, script_path=script_path, fasta_object=fasta_object, output_dir=output_dir, logfile=logfile, model_path=model_path, model_func=get_model, logfile_path=logfile_path) for query in tqdm.tqdm(list_of_regions, desc='[INFO] Creating futures for input regions')]

<function check_query at 0x7f2769a04430>
<function check_query at 0x7f2769a04430>
<function check_query at 0x7f2769a04430>


2022-11-18 16:18:05.636005: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-18 16:18:05.636087: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (thetalogin5): /proc/driver/nvidia/version does not exist
2022-11-18 16:18:05.636646: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
  0%|          | 0/6 [11:37<?, ?it/s]


<function check_query at 0x7f2769a04430>


<function check_query at 0x7f2769a04430>


<function check_query at 0x7f2769a04430>


[INFO] Creating futures for input regions: 100%|██████████| 6/6 [01:23<00:00, 13.85s/it]


In [55]:
import pstats

In [64]:
p = pstats.Stats('./script-profiled.out')
p.sort_stats('cumulative').print_stats()

Fri Nov 18 21:56:55 2022    ./script-profiled.out

         116045888 function calls (115938614 primitive calls) in 111.969 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   3399/1    0.009    0.000  111.981  111.981 {built-in method builtins.exec}
        1    0.001    0.001  111.981  111.981 enformer-predict-personalized.py:1(<module>)
        1    0.000    0.000  111.484  111.484 enformer-predict-personalized.py:23(main)
      202    0.001    0.000  111.455    0.552 enformer-predict-personalized.py:121(<genexpr>)
      201    0.030    0.000  111.390    0.554 /lus/theta-fs0/projects/covid-ct/imlab/users/temi/projects/TFXcan/enformer-predict/scripts-2/utilities/runPredictionUtilities.py:211(run_single_predictions)
      140    0.021    0.000   74.509    0.532 /lus/theta-fs0/projects/covid-ct/imlab/users/temi/projects/TFXcan/enformer-predict/scripts-2/utilities/runPredictionUtilities.py:187(enformer_predict)
      140   

In [66]:
p = pstats.Stats('./script-parsl-profiled.out')
p.sort_stats('cumulative').print_stats()

Fri Nov 18 23:41:12 2022    ./script-parsl-profiled.out

         1236173 function calls (1187955 primitive calls) in 193.601 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    937/1    0.005    0.000  193.602  193.602 {built-in method builtins.exec}
        1    0.001    0.001  193.602  193.602 enformer-predict-personalized.py:1(<module>)
        1    0.000    0.000  192.080  192.080 enformer-predict-personalized.py:41(main)
     24/5    0.000    0.000  120.515   24.103 /home/temi/miniconda3/envs/dl-tools/lib/python3.10/site-packages/typeguard/__init__.py:1030(wrapper)
        1    0.000    0.000  120.157  120.157 /lus/theta-fs0/projects/covid-ct/imlab/users/temi/projects/TFXcan/enformer-predict/scripts-2/utilities/parslConfiguration.py:2(create_parsl_configuration)
        1    0.000    0.000  120.154  120.154 /home/temi/miniconda3/envs/dl-tools/lib/python3.10/site-packages/parsl/executors/high_throughput/executor.py:1